In [2]:
from __future__ import print_function
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import enchant
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import WordNetLemmatizer 
import enchant
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

lemmatizer = WordNetLemmatizer()

In [3]:
df =pd.read_csv("../data/parsed_news_articles.csv")
df['Text'] = df['Text'].astype(str)

In [6]:
def preprocessor(text):
    stop_words = stopwords.words('english')
    d = enchant.Dict("en_GB")
    spcl= '[-,;@_!#$%^&*()<>?/\|}{~:''.+""]'
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens=[]
    for token in tokens:
        token = re.sub(spcl,'',token)
        token = re.sub('[0-9]','',token)
        if re.search('[a-zA-Z]', token):
            token = token.lower()
            if (token!='') and (d.check(token)):
                filtered_tokens.append(token)
    v = [word for word in filtered_tokens if word not in stop_words]
    stems = [lemmatizer.lemmatize(t) for t in v]
    temp = ' '.join(i for i in stems)
    return temp

In [7]:
cleanedArticles=[]

for i in df.index:
    c=preprocessor(df['Text'][i])
    cleanedArticles.append(c)

In [8]:
len(cleanedArticles)

713

In [21]:
df['Cleaned_text']=cleanedArticles

In [29]:
test_articles = pd.read_csv("../data/parsed_test_articles.csv")
cleanedTestArticles=[]

for i in test_articles.index:
    c=preprocessor(test_articles['Text'][i])
    cleanedTestArticles.append(c)
    
test_articles['Cleaned_text']=cleanedTestArticles

In [43]:
def vectorize_sim_search(query, datai):
    cos_sim=[]
    tfidf_vectorizer = TfidfVectorizer()
    dataMatrixAll = []
    similarity_dict=dict()
    for i in range(len(query)):
        query_i = query[i]
        data = [query[i]]+cleanedArticles
        lk=test_articles['Links'][i]
        similarity_dict[lk]=list()
        data = [query[i]]+datai
        dataMatrix = {}
        tfidf_matrix = tfidf_vectorizer.fit_transform(data)
        vals = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix)[0]
        for j in range(len(data)-1):
            dataMatrix[df.loc[df['Cleaned_text'] == datai[j], 'Links'].iloc[0]]=vals[j+1]
        
        key_list = list(dataMatrix.keys())
        val_list = list(dataMatrix.values())
        sorted_vals = sorted(vals,reverse=True)[1:6]
        for i in sorted_vals:
            temp=key_list[val_list.index(i)]
            similarity_dict[lk].append(temp)
    return similarity_dict

In [31]:
def sentiment_scores(text):
    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(text)
    return sentiment_dict['compound']   

In [32]:
df["sentiment"]=""
for i in range(len(cleanedArticles)):
    article_title = df['Title'].iloc[i]
    article_body = cleanedArticles[i]
    df['sentiment'][i] = sentiment_scores(article_body)

In [33]:
df.head()

,Source,Title,Text,Links,Cleaned_text,sentiment
0,The Indian Express,"Comprehensive employment, industrial policies ...",since the employment and unemployment figures ...,https://indianexpress.com/article/opinion/colu...,since employment unemployment figure always es...,0.847
1,The Indian Express,Explained: Is the push towards organised manuf...,"For a while now, it has been suggested that In...",https://indianexpress.com/article/explained/ex...,suggested unemployment woe solved boosting man...,0.9956
2,The Indian Express,It’s a wage crisis,"At a deeper level, our acceptance of self-expl...",https://indianexpress.com/article/opinion/colu...,deeper level acceptance form mental bondage de...,0.9959
3,The Indian Express,The rising insecurity,Much of the labour force shifting out of agric...,https://indianexpress.com/article/opinion/indi...,much labour force shifting agriculture found w...,0.9771
4,The Indian Express,"With India facing a job crisis, we should have...",Amazon to invest USD 1 bn in digitising Indian...,https://indianexpress.com/article/opinion/colu...,amazon invest digitising file amazon invest di...,0.7922


In [44]:
d=dict()
keys = df['Source'].unique()
for key in keys:
    d[key]=list()
for i in range(len(df)):
    d[df['Source'][i]].append(df['Cleaned_text'][i])
comparison_dict=dict()
links = list(test_articles['Links'])
for i in range (len(test_articles)):
    comparison_dict[links[i]]=list()
for source,articles in d.items():
    print(source)
    d2 = vectorize_sim_search(cleanedTestArticles,articles)
    ke=list(d2.keys())
    for key in ke:
        comparison_dict[key].append(d2[key][0])

The Indian Express
The Wire 
The Economist
India Times
Republic TV
Janta Ka Reporter
OpIndia
The Independent
NDTV
Outlook India
News Laundry
India Today
Time of India
National Herald
The Statesman
The Hindu
News18
Deccan Chronicle
Zee News
The Telegraph
First Post
DNA
The Tribune
Scroll.in
Reuters
The Financial Express
Rajya Sabha TV


In [47]:
k = list(comparison_dict.keys())
sourcesl = list(d.keys())
avg_sc = [0]*27
for link in k:
    for i in range(len(comparison_dict[link])):
        avg_sc[i]=avg_sc[i]+df.loc[df['Links'] == comparison_dict[link][i], 'sentiment'].iloc[0]
for i in range(len(avg_sc)):
    avg_sc[i]=avg_sc[i]/29
bias_list=[0]*27
for i in range(len(avg_sc)):
    if(avg_sc[i]<-0.05):
        bias_list[i]='Left'
    elif(avg_sc[i]>0.05):
        bias_list[i]='Right'
    else:
        bias_list[i]='Center'
bias_df = pd.DataFrame(list(zip(sourcesl, bias_list)),columns =['Source', 'Bias'])

In [48]:
bias_df

,Source,Bias
0,The Indian Express,Center
1,The Wire,Left
2,The Economist,Right
3,India Times,Left
4,Republic TV,Left
5,Janta Ka Reporter,Right
6,OpIndia,Right
7,The Independent,Right
8,NDTV,Left
9,Outlook India,Right
